In [ ]:
!pkill -f ngrok
!pkill -f flask  # 이건 없어도 되는 경우가 많음


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q "transformers>=4.35.3" accelerate bitsandbytes pillow


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 43.7 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
from flask import Flask, request, jsonify, render_template
from pyngrok import ngrok
import pandas as pd
import numpy as np
from PIL import Image
import torch
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from datetime import datetime
import os
import io
import json

app = Flask(__name__)



# ==============================================================================
# ⚙️ 설정 (경로 확인 필수!)
# ==============================================================================
PROJECT_PATH = "/content/drive/MyDrive/ACDT_Export"
USER_DATA_PATH = os.path.join(PROJECT_PATH, "users")
os.makedirs(USER_DATA_PATH, exist_ok=True)

# 학습된 모델 클래스
CLASS_NAMES = [
    "Bibimbap", "Bulgogi", "Chicken", "Jajangmyeon", "Jjamppong",
    "Kimchi_Stew", "Pork_Belly", "Ramen", "Tteokbokki"
]

# ==============================================================================
# 🧠 모델 및 DB 로드
# ==============================================================================
print("⏳ 모델 로딩 중...")
model_id = "llava-hf/llava-v1.6-vicuna-13b-hf"
processor = LlavaNextProcessor.from_pretrained(model_id)
llava_model = LlavaNextForConditionalGeneration.from_pretrained(
    model_id, torch_dtype=torch.float16, low_cpu_mem_usage=True, device_map="auto"
)

custom_model_path = os.path.join(PROJECT_PATH, "food_classifier_mobilenetb2.h5")
if os.path.exists(custom_model_path):
    vision_model = tf.keras.models.load_model(custom_model_path)
else:
    vision_model = None

db_path = os.path.join(PROJECT_PATH, "meal_advisor_food_db_v1.csv")
if os.path.exists(db_path):
    food_db = pd.read_csv(db_path)
else:
    food_db = pd.DataFrame()

# ==============================================================================
# 🛠️ 유저 관리 헬퍼 함수
# ==============================================================================
def load_user_data(nickname):
    path = os.path.join(USER_DATA_PATH, f"{nickname}.json")
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f: return json.load(f)
    return None

def save_user_data(nickname, data):
    path = os.path.join(USER_DATA_PATH, f"{nickname}.json")
    with open(path, "w", encoding="utf-8") as f: json.dump(data, f, ensure_ascii=False, indent=4)

# 기타 분석 함수들
def predict_food_class(image):
    if vision_model is None: return "Error", 0.0
    img_resized = image.resize((224, 224))
    img_array = img_to_array(img_resized)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    preds = vision_model.predict(img_array)
    idx = np.argmax(preds)
    return CLASS_NAMES[idx], float(np.max(preds))

def get_nutrition_info(food_name):
    if food_db.empty: return None
    row = food_db[food_db['name_en'].str.lower() == food_name.lower()]
    if not row.empty: return row.iloc[0].to_dict()
    return None

def generate_llava_response(prompt, image=None):
    if image is None: image = Image.new('RGB', (224, 224), color='white')
    inputs = processor(prompt, image, return_tensors="pt").to("cuda:0")
    output = llava_model.generate(**inputs, max_new_tokens=400)
    return processor.decode(output[0], skip_special_tokens=True).split("[/INST]")[-1].strip()

# ==============================================================================
# 🌐 API 엔드포인트
# ==============================================================================
@app.route('/')
def index(): return render_template('index.html')

# ✅ [로그인 기능] - 이게 없어서 안됐던 겁니다!
@app.route('/api/auth/login', methods=['POST'])
def login():
    data = request.json
    nickname = data.get("nickname", "").strip()
    if load_user_data(nickname): return jsonify({"success": True, "msg": "Login Successful!"})
    return jsonify({"success": False, "msg": "Nickname not found. Please Sign Up."})

# ✅ [회원가입 기능]
@app.route('/api/auth/signup', methods=['POST'])
def signup():
    data = request.json
    nickname = data.get("nickname", "").strip()
    if not nickname: return jsonify({"success": False, "msg": "Enter nickname!"})
    if load_user_data(nickname): return jsonify({"success": False, "msg": "Nickname exists!"})

    save_user_data(nickname, {"history": []})
    return jsonify({"success": True, "msg": "Account Created!"})

# [분석 및 추천]
@app.route('/api/recommend', methods=['POST'])
def api_recommend():
    nickname = request.form.get("nickname")
    user_data = load_user_data(nickname)
    if not user_data: return jsonify({"error": "Login required"}), 400

    image_file = request.files.get("image")
    img = Image.open(io.BytesIO(image_file.read())).convert('RGB')

    goal = request.form.get("goal")
    user_text = request.form.get("user_text")
    manual_food = request.form.get("manual_food")
    meal_datetime_str = request.form.get("meal_datetime", "").strip()

    food_name, conf = predict_food_class(img)
    if manual_food: food_name = manual_food

    nutri = get_nutrition_info(food_name)
    nutri_txt = f"{nutri['kcal']}kcal" if nutri else "No data"

    prompt = f"[INST] <image> User Goal: {goal}, Note: {user_text}, Food: {food_name}, Nutri: {nutri_txt}. Evaluate and recommend 2 items. [/INST]"
    advice = generate_llava_response(prompt, img)

    # 시간 저장 로직
    if meal_datetime_str: timestamp = meal_datetime_str
    else: timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M")

    try:
        h = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M").hour
        if 5 <= h < 11: m_type = "Breakfast"
        elif 11 <= h < 15: m_type = "Lunch"
        elif 15 <= h < 17: m_type = "Snack"
        elif 17 <= h < 22: m_type = "Dinner"
        else: m_type = "Late Night"
    except: m_type = "Snack"

    history_item = {
        "timestamp": timestamp,
        "meal_type": m_type,
        "food": food_name,
        "nutrition": nutri,
        "advice": advice
    }

    user_data["history"].append(history_item)
    save_user_data(nickname, user_data)

    return jsonify(history_item)

# [히스토리]
@app.route('/api/history', methods=['POST'])
def api_history():
    nickname = request.json.get("nickname")
    user_data = load_user_data(nickname)
    return jsonify(user_data["history"] if user_data else [])

# [게스트 추천]
@app.route('/api/simple_recommend', methods=['POST'])
def simple_recommend():
    data = request.json
    prompt = f"[INST] User likes: {data.get('like')}, Dislikes: {data.get('dislike')}. Recommend 3 dishes. [/INST]"
    result = generate_llava_response(prompt)
    return jsonify({"result": result})

if __name__ == '__main__':
    app.run()

Writing app.py


In [ ]:
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration

model_id = "llava-hf/llava-1.5-7b-hf"

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=dtype,
    low_cpu_mem_usage=True,
    device_map="auto",
)

processor = AutoProcessor.from_pretrained(model_id)

print("device =", device)


`torch_dtype` is deprecated! Use `dtype` instead!
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

device = cuda


In [ ]:
# ====== LLaVA 출력에서 FOOD / GOAL_SCORE / RECOMMEND / TAGS 파싱 + 핵심 LLaVA 호출 (EN-only) ======
from PIL import Image
from io import BytesIO
import os
import re
import torch

# ------------------------------------------------
# 1) 프롬프트 생성 함수 (BMI / meal_type / 프로필까지 반영)
# ------------------------------------------------
def build_prompt(
    food_label,
    nutrition_text,
    tags_from_db,
    goal,
    user_text,
    bmi=None,
    bmi_label="",
    meal_type="",
    height_cm=None,
    weight_kg=None,
):
    # 프로필 블록 구성
    profile_lines = []
    if height_cm is not None:
        profile_lines.append(f"Height: {height_cm:.1f} cm")
    if weight_kg is not None:
        profile_lines.append(f"Weight: {weight_kg:.1f} kg")
    if bmi is not None:
        profile_lines.append(f"BMI: {bmi:.1f} ({bmi_label or 'no label'})")
    if meal_type:
        profile_lines.append(f"Meal type: {meal_type}")

    profile_block = "\n".join(profile_lines) if profile_lines else "No profile info."

    prompt = f"""
USER: <image>
You are an AI meal coach for Korean university students.

USER_PROFILE:
{profile_block}

You will receive:
- FOOD_NAME: the recognized dish name (may be slightly noisy)
- NUTRITION: rough nutrition information from our table
- GOAL: the user's body goal (bulk = gain muscle, cut = lose fat, maintain = keep weight)
- USER_NOTE: a short sentence about their situation, dislikes, or allergies.

Use ALL of these (profile, goal, meal type, nutrition) to reason about whether this specific food is good for the user.

FOOD_NAME: {food_label}
NUTRITION: {nutrition_text}
BASIC_TAGS: {tags_from_db}
GOAL: {goal}
USER_NOTE: {user_text}

### Your tasks

1. Decide how suitable this food is for the user's GOAL and BMI, based on calories and the balance of protein, fat, and carbs.
2. Give clear feedback about this meal and what should be added or reduced in the next meals (for example: calories too low, almost no protein, too much fat, not enough fiber, etc.).
3. Explain what needs to be satisfied in general to reach this GOAL, and give 2–3 example dishes that fit this goal.
4. Reflect USER_NOTE (likes/dislikes/allergies) as much as possible and suggest 1–3 specific dishes that would be especially good choices for this user.
5. End with a short summary that tells the user what to do now.

### Output format (must follow exactly):

FOOD: <final food name in English>

GOAL_SCORE: <0-10> - <one short sentence explaining why this score for the current goal and BMI>

MEAL_EVAL: <2-3 sentences explaining what is strong/weak in this meal (calories, protein, fat, carbs) and what to watch out for in the next meals.>

RECOMMEND: <1-3 specific dish names in English, separated by commas>

RECOMMEND_REASON: <2-3 sentences about why these specific dishes fit the user's goal, BMI, and situation (include calories / protein / fat / carbs logic if possible).>

SUMMARY: <1-2 simple sentences summarizing what the user should do now (eat it, change portion, or choose another food).>

TAGS: <refined tags like bulk_friendly, cut_friendly, late_night, stomach_sensitive>

Do NOT add any other sections or headings.
ASSISTANT:
""".strip()
    return prompt


# ------------------------------------------------
# 2) ASSISTANT 출력에서 섹션 파싱
# ------------------------------------------------
def extract_sections(text: str, fallback_food: str = ""):
    """
    LLaVA 출력에서 각 섹션(FOOD, GOAL_SCORE, MEAL_EVAL, RECOMMEND,
    RECOMMEND_REASON, SUMMARY, TAGS)을 파싱해서 dict로 반환.
    역슬래시가 끼어 있는 'GOAL\_SCORE' 같은 것도 같이 처리.
    """
    lines = [l.strip() for l in text.splitlines() if l.strip()]

    fields = {
        "FOOD": [],
        "GOAL_SCORE": [],
        "MEAL_EVAL": [],
        "RECOMMEND": [],
        "RECOMMEND_REASON": [],
        "SUMMARY": [],
        "TAGS": [],
    }
    current = None

    for l in lines:
        # 역슬래시가 들어간 경우를 위해 정규화
        low = l.lower().replace("\\_", "_")

        if low.startswith("food:"):
            current = "FOOD"
            fields["FOOD"].append(l.split(":", 1)[1].strip())
        elif low.startswith("goal_score:"):
            current = "GOAL_SCORE"
            fields["GOAL_SCORE"].append(l.split(":", 1)[1].strip())
        elif low.startswith("meal_eval:"):
            current = "MEAL_EVAL"
            fields["MEAL_EVAL"].append(l.split(":", 1)[1].strip())
        elif low.startswith("recommend:") and not low.startswith("recommend_reason:"):
            current = "RECOMMEND"
            fields["RECOMMEND"].append(l.split(":", 1)[1].strip())
        elif low.startswith("recommend_reason:"):
            current = "RECOMMEND_REASON"
            fields["RECOMMEND_REASON"].append(l.split(":", 1)[1].strip())
        elif low.startswith("summary:"):
            current = "SUMMARY"
            fields["SUMMARY"].append(l.split(":", 1)[1].strip())
        elif low.startswith("tags:"):
            current = "TAGS"
            fields["TAGS"].append(l.split(":", 1)[1].strip())
        else:
            # 위 키워드가 아닌 줄은 직전에 해당하던 섹션에 이어붙이기
            if current is not None:
                fields[current].append(l)

    def join_field(name):
        return " ".join(fields[name]).strip()

    # FOOD 기본값 처리
    food = join_field("FOOD") or (fallback_food.strip() if fallback_food else "") or "Unknown food"

    # GOAL_SCORE 숫자 파싱
    goal_score_text = join_field("GOAL_SCORE")
    goal_score_num = None
    m = re.search(r"(\d+)", goal_score_text)
    if m:
        try:
            goal_score_num = int(m.group(1))
        except Exception:
            goal_score_num = None

    return {
        "food": food,
        "goal_score_text": goal_score_text,
        "goal_score": goal_score_num,
        "meal_eval": join_field("MEAL_EVAL"),
        "recommend": join_field("RECOMMEND"),
        "recommend_reason": join_field("RECOMMEND_REASON"),
        "summary": join_field("SUMMARY"),
        "tags": join_field("TAGS"),
    }

# ------------------------------------------------
# 3) 핵심 LLaVA 호출 함수 (이미지 경로 기반)
# ------------------------------------------------
def llava_infer_local(
    image_path,
    goal="cut",
    user_text="",
    override_food_name=None,
    bmi=None,
    bmi_label="",
    meal_type="",
    height_cm=None,
    weight_kg=None,
):
    """
    image_path: 이미지 파일 경로
    goal: 'bulk' / 'cut' / 'maintain' 등
    user_text: 한 줄 상태 설명 (상황/기분/싫어하는 음식 등)
    override_food_name: 사용자가 수동으로 입력한 음식 이름 (없으면 None)
    bmi, bmi_label, meal_type, height_cm, weight_kg: 프로필 정보 (없으면 None / "")

    return 예시 dict:
    {
        "raw": 전체 LLaVA 출력,
        "parsed": ASSISTANT 뒤 텍스트,
        "food": 최종 음식 이름,
        "food_conf": 분류기 확률 (0~1),
        "goal_score": 0~10 또는 None,
        "goal_score_text": "6/10 - ...",
        "meal_eval": "...",
        "recommend": "Dish A, Dish B",
        "recommend_reason": "...",
        "summary": "...",
        "nutrition": lookup_nutrition 결과 dict 또는 None,
        "tags": LLaVA TAGS,
        "en": 전체 assistant 텍스트 (원문 보관용)
    }
    """
    # 1) 이미지 열기
    image = Image.open(image_path).convert("RGB")

    # 2) 음식 이름 얻기 (override 있으면 그걸 우선 사용)
    if override_food_name:
        food_label = override_food_name
        food_conf = 1.0
    else:
        try:
            # 이미 노트북 상단에서 정의한 분류기 함수 사용
            food_label, food_conf = classify_food_from_pil(image)
        except Exception as e:
            print("[ERROR] classify_food_from_pil:", e)
            food_label, food_conf = "Unknown food", 0.0

    if not food_label:
        food_label = "Unknown food"

    print("[DEBUG] classifier food:", food_label, food_conf)

    # 3) 영양정보 조회 (CSV 기반)
    nut = lookup_nutrition(food_label)

    if nut is None:
        nutrition_text = "No nutrition data in table."
        tags_from_db = ""
    else:
        nutrition_text = (
            f"Estimated nutrition (per serving): "
            f"{nut.get('kcal', 0)} kcal, "
            f"{nut.get('protein_g', 0)} g protein, "
            f"{nut.get('fat_g', 0)} g fat, "
            f"{nut.get('carbs_g', 0)} g carbs."
        )

        # serving 설명 있으면 맨 앞에 붙이기
        if nut.get("serving_desc"):
            nutrition_text = f"Serving: {nut['serving_desc']}. " + nutrition_text

        # CSV에서 모은 태그 문자열 (spicy, deep_fried, late_night, ...)
        tags_from_db = nut.get("tags", "")

    # 4) LLaVA 프롬프트 생성 (EN-only + TAGS + BMI/meal_type 포함)
    prompt = build_prompt(
        food_label,
        nutrition_text,
        tags_from_db,
        goal,
        user_text,
        bmi=bmi,
        bmi_label=bmi_label,
        meal_type=meal_type,
        height_cm=height_cm,
        weight_kg=weight_kg,
    )

    # 5) LLaVA 호출 (processor, model, device는 위에서 이미 로딩되어 있다고 가정)
    inputs = processor(
        text=prompt,
        images=image,
        return_tensors="pt"
    ).to(
        model.device,
        torch.float16 if device == "cuda" else torch.float32
    )

    with torch.inference_mode():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=400,
            do_sample=False,
        )

    full_text = processor.decode(
        output_ids[0],
        skip_special_tokens=True,
    ).strip()

    # 6) 'ASSISTANT:' 뒤만 떼기
    if "ASSISTANT:" in full_text:
        assistant_text = full_text.split("ASSISTANT:", 1)[1].strip()
    else:
        assistant_text = full_text

    # 7) 섹션 파싱
    sections = extract_sections(assistant_text, fallback_food=food_label)

    print("[DEBUG] final food from LLaVA:", sections["food"])

    # 8) 결과 반환
    return {
        "raw": full_text,
        "parsed": assistant_text,
        "food": sections["food"],
        "food_conf": float(food_conf),
        "goal_score": sections["goal_score"],
        "goal_score_text": sections["goal_score_text"],
        "meal_eval": sections["meal_eval"],
        "recommend": sections["recommend"],
        "recommend_reason": sections["recommend_reason"],
        "summary": sections["summary"],
        "nutrition": nut,           # CSV dict 또는 None
        "tags": sections["tags"],   # LLaVA TAGS
        # history에서 "원문" 보고 싶을 때 사용
        "en": assistant_text,
    }


# ------------------------------------------------
# 4) Flask에서 쓰는 bytes용 wrapper
# ------------------------------------------------
def llava_infer_from_bytes(
    image_bytes,
    goal,
    user_text,
    override_food_name=None,
    bmi=None,
    bmi_label="",
    meal_type="",
    height_cm=None,
    weight_kg=None,
):
    """
    Flask에서 업로드된 이미지 바이트를 받아서
    llava_infer_local을 호출하는 보조 함수
    """
    image = Image.open(BytesIO(image_bytes)).convert("RGB")
    temp_path = "/content/temp_image_for_api.jpg"
    image.save(temp_path)

    return llava_infer_local(
        temp_path,
        goal=goal,
        user_text=user_text,
        override_food_name=override_food_name,
        bmi=bmi,
        bmi_label=bmi_label,
        meal_type=meal_type,
        height_cm=height_cm,
        weight_kg=weight_kg,
    )


# ------------------------------------------------
# 5) (선택) 단독 테스트용 코드
# ------------------------------------------------
test_image_path = "/content/drive/MyDrive/final food file/Fried Egg/Img_099_0003.jpg"
print("exists?", os.path.exists(test_image_path))
if os.path.exists(test_image_path):
    result = llava_infer_local(
        test_image_path,
        goal="cut",
        user_text="점심에 먹으려고 해요",
        bmi=21.5,
        bmi_label="normal",
        meal_type="lunch",
        height_cm=165.0,
        weight_kg=58.0,
    )
    print("=== FOOD ===", result["food"], result["food_conf"])
    print("=== GOAL_SCORE ===", result["goal_score_text"])
    print("=== MEAL_EVAL ===", result["meal_eval"])
    print("=== RECOMMEND ===", result["recommend"])
    print("=== RECOMMEND_REASON ===", result["recommend_reason"])
    print("=== SUMMARY ===", result["summary"])


<>:94: SyntaxWarning: invalid escape sequence '\_'
<>:94: SyntaxWarning: invalid escape sequence '\_'
/tmp/ipython-input-2787231136.py:94: SyntaxWarning: invalid escape sequence '\_'
  역슬래시가 끼어 있는 'GOAL\_SCORE' 같은 것도 같이 처리.


exists? True
[DEBUG] classifier food: Fried Egg 0.9432074427604675
[DEBUG] final food from LLaVA: Fried Egg
=== FOOD === Fried Egg 0.9432074427604675
=== GOAL_SCORE === 7/10 - The fried egg is a good source of protein and fat, which can help with weight loss. However, it is also high in calories, so it may not be suitable for those looking to cut.
=== MEAL_EVAL === The fried egg is a good source of protein and fat, which can help with weight loss. However, it is also high in calories, so it may not be suitable for those looking to cut.
=== RECOMMEND === Egg omelette, poached eggs, scrambled eggs
=== RECOMMEND_REASON === Egg omelettes, poached eggs, and scrambled eggs are all low in carbohydrates and high in protein, making them a good choice for those looking to cut. They can also be easily customized with various vegetables and cheeses to add more nutrients and flavor.
=== SUMMARY === Consider having an egg omelette, poached eggs, or scrambled eggs as a healthy and filling meal option

In [ ]:
# ====== Simple menu recommendation (no image) ======
from PIL import Image  # 이미 위에서 했으면 생략 가능

def build_simple_menu_prompt(like, dislike, goal, bmi=None, bmi_label=""):
    profile_lines = []
    if bmi:
        profile_lines.append(f"BMI: {bmi:.1f} ({bmi_label or 'no label'})")
    if goal:
        profile_lines.append(f"Goal: {goal} (bulk = gain muscle, cut = lose fat, maintain = keep weight)")

    profile_block = "\n".join(profile_lines) if profile_lines else "No profile info."

    prompt = f"""
USER: <image>
You are an AI meal coach for Korean university students.

USER_PROFILE:
{profile_block}

USER_PREFERENCES:
- LIKE: {like}
- DISLIKE_OR_ALLERGY: {dislike}

You will receive:
- only text about what the user likes or dislikes,
- their current body goal and BMI (if available).
You do NOT see any photo. You must imagine reasonable Korean-style meals.

### Your tasks

1. Suggest 1–3 specific dishes (in English, simple names) that fit:
   - the user's LIKE preferences,
   - avoid DISLIKE/ALLERGY,
   - and are reasonable for the GOAL (bulk / cut / maintain) and BMI.

2. Briefly explain why each dish is good, using calories and protein/fat/carbs logic
   (for example: high protein but not too oily for cut, or higher calories for bulk).

3. If the input is very vague, still give concrete dishes, not general categories.

### Output format (must follow exactly):

SIMPLE_RECOMMEND:
- <dish name 1>: <1–2 sentence reason>
- <dish name 2>: <1–2 sentence reason> (optional)
- <dish name 3>: <1–2 sentence reason> (optional)

Do NOT add any other sections or headings.
ASSISTANT:
""".strip()

    return prompt


def llava_simple_recommend(like, dislike, goal, bmi=None, bmi_label=""):
    """
    텍스트만으로 간단 식단 추천을 만드는 LLaVA 호출 함수
    """
    prompt = build_simple_menu_prompt(like, dislike, goal, bmi=bmi, bmi_label=bmi_label)

    # 더미 흰색 이미지 (이미지는 무시하라고 프롬프트에 말했음)
    dummy_image = Image.new("RGB", (10, 10), (255, 255, 255))

    inputs = processor(
        text=prompt,
        images=dummy_image,
        return_tensors="pt",
    ).to(
        model.device,
        torch.float16 if device == "cuda" else torch.float32,
    )

    with torch.inference_mode():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=400,
            do_sample=False,
        )

    full_text = processor.decode(
        output_ids[0],
        skip_special_tokens=True,
    ).strip()

    # 'ASSISTANT:' 뒤만 떼기
    if "ASSISTANT:" in full_text:
        assistant_text = full_text.split("ASSISTANT:", 1)[1].strip()
    else:
        assistant_text = full_text

    return assistant_text


히스토리 분석용 함수

In [ ]:
from PIL import Image  # 이미 임포트돼 있으면 생략해도 됨
from datetime import datetime, timedelta

def build_history_analysis_prompt(profile, meals, period_label="last 7 days"):
    """
    profile: dict {height_cm, weight_kg, bmi, bmi_label}
    meals:   히스토리에서 뽑은 서브셋 list[dict]
    period_label: "last 7 days", "last 30 days", "2025-12-01 to 2025-12-10" 이런 문자열
    """

    height_cm = profile.get("height_cm")
    weight_kg = profile.get("weight_kg")
    bmi = profile.get("bmi")
    bmi_label = profile.get("bmi_label", "")

    # --- 프로필 블록 텍스트 ---
    profile_lines = []
    if height_cm:
        profile_lines.append(f"Height: {height_cm:.1f} cm")
    if weight_kg:
        profile_lines.append(f"Weight: {weight_kg:.1f} kg")
    if bmi:
        profile_lines.append(f"BMI: {bmi:.1f} ({bmi_label or 'no label'})")

    if not profile_lines:
        profile_block = "No profile info."
    else:
        profile_block = "\n".join(profile_lines)

    # --- 숫자 요약 계산 (평균 kcal / P / F / C, late_night, deep_fried 등) ---
    total_kcal = total_p = total_f = total_c = 0.0
    nut_count = 0
    late_night = 0
    deep_fried = 0
    cut_cnt = bulk_cnt = maintain_cnt = 0

    meal_lines = []
    for i, m in enumerate(meals, start=1):
        ts = m.get("timestamp", "")
        goal = m.get("goal", "")
        meal_type = m.get("meal_type", "")
        food = m.get("food", "Unknown food")
        tags = (m.get("tags") or "").lower()
        nut = m.get("nutrition") or {}

        kcal = nut.get("kcal")
        p = nut.get("protein_g")
        f = nut.get("fat_g")
        c = nut.get("carbs_g")

        if isinstance(kcal, (int, float)):
            total_kcal += kcal
            total_p += float(p or 0)
            total_f += float(f or 0)
            total_c += float(c or 0)
            nut_count += 1

        if "late_night" in tags:
            late_night += 1
        if "deep_fried" in tags:
            deep_fried += 1

        if goal == "cut":
            cut_cnt += 1
        elif goal == "bulk":
            bulk_cnt += 1
        elif goal == "maintain":
            maintain_cnt += 1

        # 한 끼 로그 한 줄 (LLaVA가 보기 쉬운 형태)
        line_parts = [f"{i}) {ts}"]
        if meal_type:
            line_parts.append(f"[meal_type={meal_type}]")
        if goal:
            line_parts.append(f"[goal={goal}]")
        line_parts.append(food)
        if isinstance(kcal, (int, float)):
            line_parts.append(f"- {kcal} kcal, {p}g P / {f}g F / {c}g C")
        if tags:
            line_parts.append(f"(tags: {tags})")

        meal_lines.append(" ".join(str(x) for x in line_parts if x))

    if nut_count > 0:
        avg_kcal = total_kcal / nut_count
        avg_p = total_p / nut_count
        avg_f = total_f / nut_count
        avg_c = total_c / nut_count
    else:
        avg_kcal = avg_p = avg_f = avg_c = 0.0

    numeric_summary = [
        f"Total meals: {len(meals)}",
        f"Meals with nutrition data: {nut_count}",
    ]
    if nut_count > 0:
        numeric_summary.append(
            f"Average per meal: {avg_kcal:.0f} kcal, {avg_p:.1f} g protein, "
            f"{avg_f:.1f} g fat, {avg_c:.1f} g carbs"
        )

    if late_night or deep_fried:
        numeric_summary.append(
            f"Late-night meals (tagged): {late_night}, Deep-fried meals (tagged): {deep_fried}"
        )

    goal_total = cut_cnt + bulk_cnt + maintain_cnt
    if goal_total > 0:
        numeric_summary.append(
            f"Goal usage: cut {cut_cnt}, bulk {bulk_cnt}, maintain {maintain_cnt}"
        )

    numeric_block = "\n".join(numeric_summary) if numeric_summary else "No numeric summary."

    meals_block = "\n".join(meal_lines) if meal_lines else "No meals in this period."

    # --- LLaVA 프롬프트 ---
    prompt = f"""
USER: <image>
You are an AI nutrition and meal-planning coach for Korean university students.

USER_PROFILE:
{profile_block}

PERIOD:
{period_label}

NUMERIC_SUMMARY:
{numeric_block}

MEAL_LOG (each line is one meal):
{meals_block}

### Your tasks

1. Analyze the user's current eating pattern in this period, including:
   - overall calorie level,
   - protein / fat / carbs balance,
   - late-night / deep-fried habits,
   - consistency with their goals (cut / bulk / maintain),
   - approximate risk of gaining too much weight or losing too much weight.

2. Describe in clear, direct language what will likely happen if this eating pattern continues
   (for example: higher risk of weight gain, unstable energy, difficulty building muscle, etc.).

3. Give 3–6 concrete habit improvements based on this pattern
   (for example: move more calories to breakfast, add protein to every meal, reduce fried foods at night, etc.).

4. Propose a ONE_DAY_MEAL_PLAN based on this analysis and the user's profile:
   - 2 or 3 main meals (breakfast / lunch / dinner),
   - 0–2 snacks.
   Each meal or snack must have:
   - a specific dish name (in English, simple),
   - a short reason,
   - a short note about calories and protein/fat/carbs role.
   If the pattern suggests a need for strong calorie deficit, you may suggest only 2 main meals and maybe 1 snack.
   If the pattern suggests under-eating or too much weight loss, you may suggest 3 main meals and 1–2 snacks.

### Output format (must follow exactly):

ANALYSIS:
<2–4 short paragraphs analyzing the pattern and risk of weight gain or excessive weight loss.>

HABIT_ADVICE:
- <bullet point 1>
- <bullet point 2>
- <bullet point 3>
(you may add up to 6 bullet points total)

ONE_DAY_MEAL_PLAN:
- BREAKFAST: <dish name>
  REASON: <1–2 sentences>
  MACRO_ROLE: <short sentence about calories / protein / fat / carbs role>
- LUNCH: <dish name>
  REASON: <1–2 sentences>
  MACRO_ROLE: <short sentence>

- DINNER: <dish name>
  REASON: <1–2 sentences>
  MACRO_ROLE: <short sentence>

SNACKS:
- <snack name 1 + 1 short reason> (or write "None" if you do not recommend snacks)
- <snack name 2 + 1 short reason> (optional)

Do NOT add any other sections or headings.
ASSISTANT:
""".strip()

    return prompt


In [ ]:
def llava_analyze_history_with_llava(profile, meals, period_label="last 7 days"):
    """
    profile: {height_cm, weight_kg, bmi, bmi_label}
    meals:   history 서브셋 list[dict]
    return:  {"text": ..., "raw": ...}
    """
    prompt = build_history_analysis_prompt(profile, meals, period_label)

    # 더미 흰색 이미지 (텍스트만 쓸 거라 "이미지는 무시하라"고 프롬프트에 썼으니 상관 없음)
    dummy_image = Image.new("RGB", (10, 10), (255, 255, 255))

    inputs = processor(
        text=prompt,
        images=dummy_image,
        return_tensors="pt"
    ).to(
        model.device,
        torch.float16 if device == "cuda" else torch.float32
    )

    with torch.inference_mode():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=600,
            do_sample=False,
        )

    full_text = processor.decode(
        output_ids[0],
        skip_special_tokens=True,
    ).strip()

    # 여기서는 'ASSISTANT:' 뒤만 떼도 되고, 안 떼도 됨
    if "ASSISTANT:" in full_text:
        assistant_text = full_text.split("ASSISTANT:", 1)[1].strip()
    else:
        assistant_text = full_text

    return {
        "raw": full_text,
        "text": assistant_text,
    }


In [ ]:
test_image_path = "/content/drive/MyDrive/final food file/Fried Egg/Img_099_0003.jpg"

import os
print("exists?", os.path.exists(test_image_path))

result = llava_infer_local(
    test_image_path,
    goal="cut",
    user_text="점심에 먹으려고 해요"
)

print("=== CLEAN EN ===")
print(result["en"])



exists? True
[DEBUG] classifier food: Fried Egg 0.9432074427604675
[DEBUG] final food from LLaVA: Fried Egg
=== CLEAN EN ===
FOOD: Fried Egg

GOAL\_SCORE: 7/10 - The fried egg is a good source of protein and fat, which can help with weight loss. However, it is also high in calories, so it should be consumed in moderation.

MEAL\_EVAL: The fried egg is a good source of protein and fat, which can help with weight loss. However, it is also high in calories, so it should be consumed in moderation.

RECOMMEND: Egg omelette, poached eggs, scrambled eggs

RECOMMEND\_REASON: Egg omelettes, poached eggs, and scrambled eggs are all low in carbohydrates and high in protein, making them a good choice for a cut-friendly meal. They can be prepared with a variety of vegetables and other ingredients to add variety and nutrition.

SUMMARY: Enjoy the fried egg in moderation as part of a balanced diet, and consider preparing egg omelettes, poached eggs, or scrambled eggs with a variety of vegetables and 

In [ ]:
print("[DEBUG result]", result)
print("[DEBUG food_label]", result.get("food"), result.get("food_conf"))


[DEBUG result] {'raw': "USER:  \nYou are an AI meal coach for Korean university students.\n\nUSER_PROFILE:\nNo profile info.\n\nYou will receive:\n- FOOD_NAME: the recognized dish name (may be slightly noisy)\n- NUTRITION: rough nutrition information from our table\n- GOAL: the user's body goal (bulk = gain muscle, cut = lose fat, maintain = keep weight)\n- USER_NOTE: a short sentence about their situation, dislikes, or allergies.\n\nUse ALL of these (profile, goal, meal type, nutrition) to reason about whether this specific food is good for the user.\n\nFOOD_NAME: Fried Egg\nNUTRITION: Serving: 1 egg. Estimated nutrition (per serving): 90.0 kcal, 6.0 g protein, 7.0 g fat, 0.0 g carbs.\nBASIC_TAGS: late_night, cut_friendly, low_carb, low_fat, low_sugar, noodle, contains_egg, contains_milk\nGOAL: cut\nUSER_NOTE: 점심에 먹으려고 해요\n\n### Your tasks\n\n1. Decide how suitable this food is for the user's GOAL and BMI, based on calories and the balance of protein, fat, and carbs.\n2. Give clear fe

In [ ]:
from flask import Flask, request, jsonify, Response
from flask_cors import CORS
from datetime import datetime, timedelta
from io import BytesIO
from PIL import Image
from threading import Thread
import json
import math
import os
import random
import re  # ✅ 추가

# ✅ KST timezone (Colab Python 3.9+)
try:
    from zoneinfo import ZoneInfo
    KST = ZoneInfo("Asia/Seoul")
except Exception:
    KST = None  # fallback (naive datetime)

VERSION = "v3rec_en_override_2025-12-15_reason_glucosefix"

app = Flask(__name__)
CORS(app)

# =========================
# ✅ Paths / Storage
# =========================
PROJECT_PATH = os.getcwd()
USER_DATA_PATH = os.path.join(PROJECT_PATH, "users")
os.makedirs(USER_DATA_PATH, exist_ok=True)

# 서버 메모리 히스토리 (닉네임 안 쓰는 경우 기본 저장)
history = []

def load_user_data(nickname: str):
    if not nickname:
        return None
    path = os.path.join(USER_DATA_PATH, f"{nickname}.json")
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    return None

def save_user_data(nickname: str, data: dict):
    if not nickname:
        return
    path = os.path.join(USER_DATA_PATH, f"{nickname}.json")
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# =========================
# ✅ (NEW) Force 3 non-overlapping recommendations
# =========================
def _norm(s: str) -> str:
    s = (str(s or "").strip()).lower()
    s = re.sub(r"[^a-z0-9가-힣]+", "", s)  # ✅ 공백/기호 제거로 비교 강화
    return s

def _parse_recommend_any(x):
    """
    llava가 recommend를 string/리스트/블릿 형태로 주더라도
    최대한 리스트로 정리해주는 파서
    """
    if x is None:
        return []
    if isinstance(x, list):
        return [str(v).strip() for v in x if str(v).strip()]
    if isinstance(x, str):
        s = x.replace("•", "\n").replace(" - ", "\n").replace("–", "\n")
        s = s.replace("|", ",").replace(";", ",")
        parts = []
        for line in s.splitlines():
            line = line.strip()
            if not line:
                continue
            # "1) dish" / "1. dish" 정리
            line = re.sub(r"^\s*\d+[\)\.\-]\s*", "", line).strip()
            if not line:
                continue

            # "- Dish: ..." 형태면 ':' 앞만 가져오기도
            if ":" in line and len(line.split(":")[0]) <= 60:
                head = line.split(":")[0].strip()
                parts.append(head if head else line)
            else:
                parts.append(line)

        out = []
        for p in parts:
            if "," in p:
                out += [t.strip() for t in p.split(",") if t.strip()]
            else:
                out.append(p)
        return out
    return [str(x).strip()] if str(x).strip() else []

def _find_nutrition_source():
    """
    노트북에 이미 로드되어 있는 nutrition 테이블/df를 자동으로 찾아보는 함수.
    - DataFrame 비슷한 객체: iterrows + columns
    - dict 형태 DB
    못 찾으면 None 반환
    """
    df_keys = [
        "nutrition_df", "NUTRITION_DF", "df_nutrition", "FOOD_NUTRITION_DF",
        "NUTRITION_TABLE_DF", "food_nutrition_df"
    ]
    for k in df_keys:
        obj = globals().get(k)
        if obj is not None and hasattr(obj, "iterrows") and hasattr(obj, "columns"):
            return ("df", obj)

    dict_keys = [
        "nutrition_table", "NUTRITION_TABLE", "NUTRITION_DB", "FOOD_DB",
        "FOOD_NUTRITION", "food_nutrition"
    ]
    for k in dict_keys:
        obj = globals().get(k)
        if isinstance(obj, dict) and len(obj) > 0:
            return ("dict", obj)

    return (None, None)

def _build_candidates_from_source(source_type, source_obj):
    """
    candidates: [{name, kcal, protein, fat, carb, tags}]
    """
    cands = []

    if source_type == "df":
        df = source_obj
        cols = list(getattr(df, "columns", []))

        def pick_col(options):
            for c in options:
                if c in cols:
                    return c
            return None

        name_col = pick_col(["food_name", "name", "Food", "dish", "label", "class_name"])
        kcal_col = pick_col(["kcal", "calories", "energy_kcal"])
        p_col    = pick_col(["protein_g", "protein"])
        f_col    = pick_col(["fat_g", "fat"])
        c_col    = pick_col(["carbs_g", "carb", "carbohydrate_g"])
        tags_col = pick_col(["tags", "tag"])

        if not name_col:
            return []

        for _, r in df.iterrows():
            name = str(r.get(name_col) or "").strip()
            if not name:
                continue
            cands.append({
                "name": name,
                "kcal": float(r.get(kcal_col) or 0) if kcal_col else 0.0,
                "protein": float(r.get(p_col) or 0) if p_col else 0.0,
                "fat": float(r.get(f_col) or 0) if f_col else 0.0,
                "carb": float(r.get(c_col) or 0) if c_col else 0.0,
                "tags": str(r.get(tags_col) or "").lower() if tags_col else "",
            })

    elif source_type == "dict":
        db = source_obj
        for name, r in db.items():
            if not name:
                continue
            if isinstance(r, dict):
                cands.append({
                    "name": str(name).strip(),
                    "kcal": float(r.get("kcal") or 0),
                    "protein": float(r.get("protein_g") or r.get("protein") or 0),
                    "fat": float(r.get("fat_g") or r.get("fat") or 0),
                    "carb": float(r.get("carbs_g") or r.get("carb") or 0),
                    "tags": str(r.get("tags") or "").lower(),
                })
    return cands

def pick_top3_recs(recognized_food: str, goal: str, tags_list=None, k=3):
    """
    ✅ 항상 겹치지 않는 3개를 만들어서 반환
    - 가능한 경우: nutrition_df / nutrition_table에서 점수 기반 top3
    - 없으면: SIMPLE_MENU_DB에서라도 3개 채움
    """
    recognized_food_n = _norm(recognized_food)
    goal = (goal or "").lower()
    tags_list = tags_list or []

    source_type, source_obj = _find_nutrition_source()
    candidates = _build_candidates_from_source(source_type, source_obj) if source_type else []

    if not candidates:
        # fallback: SIMPLE_MENU_DB
        for m in SIMPLE_MENU_DB:
            candidates.append({
                "name": m["name"],
                "kcal": 0.0,
                "protein": 0.0,
                "fat": 0.0,
                "carb": 0.0,
                "tags": " ".join(m.get("tags", [])).lower()
            })

    def score(c):
        kcal = c.get("kcal", 0.0) or 0.0
        p = c.get("protein", 0.0) or 0.0
        f = c.get("fat", 0.0) or 0.0
        carb = c.get("carb", 0.0) or 0.0
        t = (c.get("tags") or "")

        bonus = 0.0
        if goal == "cut":
            if "cut" in t or "light" in t:
                bonus += 2.0
            return (p * 2.0) - (kcal / 280.0) - (f / 18.0) + bonus
        elif goal == "bulk":
            if "bulk" in t or "high_protein" in t:
                bonus += 2.0
            return (p * 1.6) + (carb / 35.0) + (kcal / 1200.0) + bonus
        else:
            if "balanced" in t:
                bonus += 1.0
            return (p * 1.8) - (abs(kcal - 650.0) / 500.0) + bonus

    candidates = sorted(candidates, key=score, reverse=True)

    picked = []
    seen = set()
    for c in candidates:
        name = str(c.get("name") or "").strip()
        if not name:
            continue
        n = _norm(name)
        if n == recognized_food_n:
            continue
        if n in seen:
            continue
        seen.add(n)
        picked.append(name)
        if len(picked) == k:
            break

    if len(picked) < k:
        rest = []
        for c in candidates:
            name = str(c.get("name") or "").strip()
            n = _norm(name)
            if not name:
                continue
            if n == recognized_food_n or n in seen:
                continue
            rest.append(name)
        random.shuffle(rest)
        for name in rest:
            picked.append(name)
            if len(picked) == k:
                break

    while len(picked) < k:
        picked.append("Add a simple protein side (eggs / tofu / chicken breast)")

    return picked[:k]

# =========================
# ✅ Energy curve helpers
# =========================
def _clamp(v, lo=0, hi=100):
    try:
        v = float(v)
    except Exception:
        v = 0.0
    return max(lo, min(hi, v))

def _gauss(t, mu, sigma):
    if sigma <= 0:
        return 0.0
    z = (t - mu) / sigma
    return math.exp(-0.5 * z * z)

def _risk_label(r):
    if r >= 0.70:
        return "HIGH"
    if r >= 0.40:
        return "MED"
    return "LOW"

def _parse_meal_datetime_to_iso(meal_datetime_str: str):
    if meal_datetime_str:
        try:
            dt = datetime.fromisoformat(meal_datetime_str)
            if dt.tzinfo is None and KST is not None:
                dt = dt.replace(tzinfo=KST)
            return dt.isoformat(timespec="seconds")
        except Exception:
            pass

    if KST is not None:
        return datetime.now(KST).isoformat(timespec="seconds")
    return datetime.now().isoformat(timespec="seconds")

# =========================
# ✅ (PATCH) Energy payload: food-aware spike + peak timing + glucose_curve key
# =========================
def make_energy_payload(nutrition: dict, tags: list = None, meal_type: str = "", goal: str = "", user_text: str = "", start_iso: str = ""):
    if not nutrition or not isinstance(nutrition, dict) or nutrition.get("kcal") is None:
        return {}

    tags = tags or []
    tags_lower = [str(t).lower() for t in tags]
    txt = (user_text or "").lower()

    kcal = float(nutrition.get("kcal", 0) or 0)
    carbs = float(nutrition.get("carbs_g", 0) or 0)
    protein = float(nutrition.get("protein_g", 0) or 0)
    fat = float(nutrition.get("fat_g", 0) or 0)

    # ✅ food hint (api_recommend에서 _food_label 주입)
    food_hint = str(nutrition.get("_food_label", "") or "").lower()

    carb_kcal = max(0.0, carbs * 4.0)
    prot_kcal = max(0.0, protein * 4.0)
    fat_kcal  = max(0.0, fat * 9.0)
    macro_sum = carb_kcal + prot_kcal + fat_kcal

    if macro_sum > 1e-6:
        carb_ratio = carb_kcal / macro_sum
        prot_ratio = prot_kcal / macro_sum
        fat_ratio  = fat_kcal  / macro_sum
    else:
        carb_ratio = prot_ratio = fat_ratio = 0.0

    # ✅ spike modifier (음식/텍스트/태그 반영 → 음식마다 곡선 달라짐)
    sugar_keywords = ["sweet", "dessert", "cake", "cookie", "soda", "juice", "boba", "ice cream", "chocolate", "donut"]
    refined_carb_keywords = ["bread", "toast", "sandwich", "noodle", "ramen", "ramyeon", "pasta", "rice", "bap", "tteok", "tteokbokki"]
    low_gi_keywords = ["salad", "vegetable", "veggie", "tofu", "egg", "beans", "yogurt", "chicken breast"]

    mod = 0.0
    if any(k in food_hint for k in sugar_keywords): mod += 0.22
    if any(k in food_hint for k in ["bread", "toast", "sandwich"]): mod += 0.16
    if any(k in food_hint for k in ["rice", "bap", "tteok", "tteokbokki"]): mod += 0.10
    if any(k in food_hint for k in ["noodle", "ramen", "ramyeon", "jjamppong", "jjajang", "udon", "pasta"]): mod += 0.08
    if any(k in food_hint for k in low_gi_keywords): mod -= 0.10

    if any(t in tags_lower for t in ["deep_fried", "high_calorie"]): mod += 0.05
    if any(t in tags_lower for t in ["light", "salad", "veggie", "balanced"]): mod -= 0.05
    if any(k in txt for k in sugar_keywords): mod += 0.10

    # ✅ risk score (0~1)
    base_risk = (0.45 * carb_ratio) + (0.35 * min(carbs / 90.0, 1.0)) + (0.20 * min(kcal / 900.0, 1.0))
    damp = (0.25 * min(protein / 35.0, 1.0)) + (0.20 * min(fat / 30.0, 1.0))
    gly_score = _clamp(base_risk - damp + mod, 0, 1)
    risk_label = _risk_label(gly_score)

    # base energy
    base_energy = 55.0
    if meal_type == "breakfast":
        base_energy = 58.0
    elif meal_type == "lunch":
        base_energy = 56.0
    elif meal_type == "dinner":
        base_energy = 52.0
    elif meal_type == "late_night":
        base_energy = 47.0

    goal_l = (goal or "").lower().strip()
    if goal_l == "cut":
        base_energy -= 1.5
    elif goal_l == "bulk":
        base_energy += 1.0

    if "late_night" in tags_lower:
        base_energy -= 2.0

    # ✅ peak timing (fat delays, sugar/refined makes earlier)
    fat_delay = 22.0 * fat_ratio
    earlier = 18.0 * max(mod, 0.0)
    carb_peak = _clamp(55.0 + fat_delay - earlier, 25, 120)
    carb_w    = 22.0 + 28.0 * fat_ratio
    prot_peak = _clamp(95.0 + 0.8 * fat_delay, 60, 170)
    prot_w    = 35.0 + 18.0 * fat_ratio
    fat_peak  = _clamp(145.0 + 1.2 * fat_delay, 90, 220)
    fat_w     = 55.0 + 30.0 * fat_ratio

    total_amp = _clamp(12.0 + kcal / 55.0, 12, 38)
    carb_amp  = total_amp * (0.55 + 0.45 * carb_ratio)
    prot_amp  = total_amp * (0.18 + 0.55 * prot_ratio) * 0.85
    fat_amp   = total_amp * (0.10 + 0.65 * fat_ratio)  * 0.75

    dip_peak = carb_peak + (90.0 + 25.0 * gly_score)
    dip_w    = 32.0
    dip_amp  = 18.0 * gly_score

    horizon = 240
    step = 5

    points = []
    glucose_points = []

    base_gluc = 45.0
    gluc_amp = _clamp(12.0 + 55.0 * gly_score, 12, 65)

    for t in range(0, horizon + 1, step):
        carb_b = carb_amp * _gauss(t, carb_peak, carb_w)
        prot_b = prot_amp * _gauss(t, prot_peak, prot_w)
        fat_b  = fat_amp  * _gauss(t, fat_peak,  fat_w)
        dip    = dip_amp  * _gauss(t, dip_peak,  dip_w)

        energy = base_energy + 0.70 * carb_b + 0.85 * prot_b + 0.60 * fat_b - dip
        energy = _clamp(energy, 0, 100)

        glucose = base_gluc + gluc_amp * _gauss(t, carb_peak, carb_w * 0.92) - (9.0 * gly_score) * _gauss(t, dip_peak, dip_w * 1.1)
        glucose = _clamp(glucose, 0, 100)

        points.append({"t": float(t), "y": float(round(energy, 2))})
        glucose_points.append({"t": float(t), "y": float(round(glucose, 2))})

    # ✅ glucose peak detection
    g_peak = max(glucose_points, key=lambda p: p["y"]) if glucose_points else {"t": carb_peak, "y": 0}
    glucose_peak_time_min = float(g_peak.get("t", carb_peak))
    glucose_peak_value = float(g_peak.get("y", 0))

    # microcopy
    micro = "Expected curve (next 4 hours). Solid: energy · Dashed: blood sugar."
    if risk_label == "HIGH":
        micro = "Fast rise + higher spike risk. Add protein/fat/fiber next time for steadier energy."
    elif risk_label == "MED":
        micro = "Moderate rise. A small protein side can make it steadier."
    else:
        micro = "Steadier pattern. Current macro balance may be helping."

    events = [
        {"t": 10, "emoji": "🍽️", "title": "Digestion", "short": "Start", "text": "Digestion begins; energy and glucose may start shifting."},
        {"t": glucose_peak_time_min, "emoji": "📈", "title": "Glucose peak", "short": "Peak", "text": f"Estimated blood sugar peak around here. Risk: {risk_label}."},
        {"t": 120, "emoji": "🏃", "title": "Workout window", "short": "Workout", "text": "Often a good window for light-to-moderate activity."},
    ]
    if gly_score >= 0.40:
        events.append({"t": max(20, glucose_peak_time_min - 15), "emoji": "⚠️", "title": "Spike window", "short": "Spike", "text": "This window often shows the fastest rise. Protein/fiber helps next time."})
    if gly_score >= 0.60:
        events.append({"t": dip_peak, "emoji": "🥱", "title": "Dip risk", "short": "Dip", "text": "A dip can follow a fast spike. Water + protein may stabilize."})
    if "late_night" in tags_lower or meal_type == "late_night":
        events.append({"t": 210, "emoji": "🌙", "title": "Sleep sensitivity", "short": "Sleep", "text": "Late meals can affect sleep quality for some people."})

    events = sorted(events, key=lambda e: float(e.get("t", 0)))

    glucose_metrics = {
        "kcal": kcal,
        "carbs_g": carbs,
        "protein_g": protein,
        "fat_g": fat,
        "carb_ratio": carb_ratio,
        "prot_ratio": prot_ratio,
        "fat_ratio": fat_ratio,
        "glycemic_risk": risk_label,
        "glycemic_risk_score": float(round(gly_score, 3)),
        "glucose_peak_time_min": float(round(glucose_peak_time_min, 1)),
        "glucose_peak_value": float(round(glucose_peak_value, 1)),
    }

    return {
        "microcopy": micro,
        "curve": {
            "horizon_min": horizon,
            "y_min": 0,
            "y_max": 100,
            "start_iso": start_iso or "",
            "points": points,
            "glucose_points": glucose_points,  # ✅ 기존 호환 유지
            "metrics": {
                "kcal": kcal,
                "carbs_g": carbs,
                "protein_g": protein,
                "fat_g": fat,
                "carb_ratio": carb_ratio,
                "prot_ratio": prot_ratio,
                "fat_ratio": fat_ratio,
                "glycemic_risk": risk_label,
            },
        },
        # ✅ 프론트가 우선으로 쓰기 좋은 키 (점선 강제 대체 방지)
        "glucose_curve": {
            "horizon_min": horizon,
            "y_min": 0,
            "y_max": 100,
            "start_iso": start_iso or "",
            "points": glucose_points,
            "metrics": glucose_metrics,
        },
        "events": events,
    }

# =========================
# ✅ (PATCH) Reason text builders (goal + macro + spike 기반)
# =========================
def _macro_line(kcal, p, f, c):
    return f"{int(kcal)} kcal · {int(p)}g P / {int(f)}g F / {int(c)}g C"

def build_goal_reason_text(food_label: str, goal: str, nut: dict, rec_list: list, glucose_metrics: dict):
    goal_l = (goal or "").lower().strip()
    goal_name = goal_l if goal_l in ["cut", "bulk"] else "maintain"

    kcal = float((nut or {}).get("kcal") or 0)
    p = float((nut or {}).get("protein_g") or 0)
    c = float((nut or {}).get("carbs_g") or 0)
    f = float((nut or {}).get("fat_g") or 0)

    notes = []
    if goal_name == "cut":
        if kcal >= 750: notes.append("calories are quite high for cutting")
        if c >= 80: notes.append("carbs are high, so portion control matters")
        if p < 22: notes.append("protein is a bit low for satiety/muscle protection")
    elif goal_name == "bulk":
        if kcal <= 600: notes.append("calories may be low for bulking")
        if p < 25: notes.append("protein may be low for muscle gain")
    else:
        if p < 20: notes.append("protein is a bit low for stable meals")
        if c >= 90: notes.append("carbs are on the higher side (portion matters)")

    risk = (glucose_metrics or {}).get("glycemic_risk", "")
    peak_t = (glucose_metrics or {}).get("glucose_peak_time_min", None)
    spike_line = ""
    if peak_t is not None and risk:
        spike_line = f"Estimated blood sugar peak ~{int(float(peak_t))} min after the meal (risk: {risk})."

    # ✅ meal evaluator (네가 원한 문장 구조)
    meal_eval = (
        f"The detected dish is **{food_label}**. "
        f"Macros are about {_macro_line(kcal, p, f, c)}. "
        f"Your goal is **{goal_name}**. "
    )
    if notes:
        meal_eval += "For your goal, the key improvement is: " + "; ".join(notes) + ". "
    if spike_line:
        meal_eval += spike_line

    # ✅ why these dishes (네 음식 -> 목표 -> 부족 -> 대체안)
    why_lines = []
    why_lines.append(f"Because your meal was **{food_label}** and your goal is **{goal_name}**, I checked your macros first.")
    why_lines.append(f"Your meal: {_macro_line(kcal, p, f, c)}.")
    if notes:
        why_lines.append("What to improve for your goal: " + "; ".join(notes) + ".")
    if spike_line:
        why_lines.append(spike_line)
    why_lines.append("So I recommended these 3 alternatives to address those points (and avoid repeating the same dish):")

    def dish_intent(dish: str):
        d = (dish or "").lower()
        intents = []

        # 단백질 보강
        if p < 22 and goal_name in ["cut", "maintain"]:
            if any(k in d for k in ["chicken", "tofu", "egg", "salad"]):
                intents.append("adds lean protein to help satiety")

        if goal_name == "bulk":
            if any(k in d for k in ["chicken", "pork", "beef", "high_protein"]):
                intents.append("supports higher protein for bulking")

        # 탄수/섬유 균형
        if c >= 80:
            if any(k in d for k in ["salad", "vegetable", "veggie"]):
                intents.append("adds fiber to slow the spike and balance carbs")
            if "bibimbap" in d:
                intents.append("more vegetables → better balance than heavy noodles")
            if any(k in d for k in ["cold noodles", "naengmyeon"]):
                intents.append("often lighter than heavy noodle soups")

        # 칼로리
        if goal_name == "cut" and kcal >= 750:
            if any(k in d for k in ["salad", "light", "cold noodles", "naengmyeon"]):
                intents.append("typically lighter, easier to keep calories controlled")

        if not intents:
            intents.append("a different option that matches your goal better")

        # 중복 제거
        uniq = []
        seen = set()
        for x in intents:
            if x not in seen:
                uniq.append(x)
                seen.add(x)
        return "; ".join(uniq)

    bullets = []
    for dish in rec_list[:3]:
        bullets.append(f"- {dish}: {dish_intent(dish)}.")

    recommend_reason_text = "\n".join(why_lines + bullets)
    summary_text = "Try one of these next meals:\n" + "\n".join([f"- {x}" for x in rec_list[:3]])

    return meal_eval, recommend_reason_text, summary_text

# =========================
# ✅ basic APIs
# =========================
@app.route("/api/test")
def api_test():
    return jsonify({"msg": "server running", "version": VERSION})

# --- 이미지 바이트 -> 로컬 LLaVA 호출용 보조 함수 ---
def llava_infer_from_bytes(
    image_bytes,
    goal,
    user_text,
    override_food_name=None,
    bmi=None,
    bmi_label="",
    meal_type="",
    height_cm=None,
    weight_kg=None,
):
    if "llava_infer_local" not in globals():
        raise RuntimeError("llava_infer_local() is not defined in this notebook. Please run the cell that defines it first.")

    image = Image.open(BytesIO(image_bytes)).convert("RGB")
    temp_path = os.path.join(PROJECT_PATH, "temp_image_for_api.jpg")
    image.save(temp_path)

    return llava_infer_local(
        temp_path,
        goal=goal,
        user_text=user_text,
        override_food_name=override_food_name,
        bmi=bmi,
        bmi_label=bmi_label,
        meal_type=meal_type,
        height_cm=height_cm,
        weight_kg=weight_kg,
    )

# =========================
# ✅ Recommend API
# =========================
@app.route("/api/recommend", methods=["POST"])
def api_recommend():
    image_file = request.files.get("image")
    goal = request.form.get("goal", "")
    user_text = request.form.get("user_text", "")
    manual_food = (request.form.get("manual_food", "") or "").strip()
    nickname = (request.form.get("nickname", "") or "").strip()

    height_cm = request.form.get("height_cm", type=float)
    weight_kg = request.form.get("weight_kg", type=float)
    meal_type = request.form.get("meal_type", "") or ""
    meal_datetime = (request.form.get("meal_datetime", "") or "").strip()

    final_timestamp = _parse_meal_datetime_to_iso(meal_datetime)

    bmi = None
    bmi_label = ""
    if height_cm and weight_kg and height_cm > 0:
        h_m = height_cm / 100.0
        bmi = weight_kg / (h_m * h_m)
        if bmi < 18.5:
            bmi_label = "underweight"
        elif bmi < 23:
            bmi_label = "normal"
        elif bmi < 25:
            bmi_label = "overweight (Asian cutoff)"
        else:
            bmi_label = "obese (Asian cutoff)"

    if image_file is None:
        return jsonify({"error": "image field 'image' is required"}), 400

    image_bytes = image_file.read()

    try:
        result = llava_infer_from_bytes(
            image_bytes,
            goal,
            user_text,
            override_food_name=manual_food or None,
            bmi=bmi,
            bmi_label=bmi_label,
            meal_type=meal_type,
            height_cm=height_cm,
            weight_kg=weight_kg,
        )
    except Exception as e:
        print("[ERROR] llava_infer_from_bytes:", e)
        return jsonify({"error": str(e)}), 500

    food_label = (
        result.get("food")
        or result.get("label")
        or result.get("food_name")
        or result.get("class_name")
        or "Unknown food"
    )
    food_conf = float(result.get("food_conf") or 0.0)
    nut = result.get("nutrition") or {}

    # tags 정규화
    raw_tags = result.get("tags") or []
    tags_list = []
    if isinstance(raw_tags, list):
        tags_list = [str(t).strip() for t in raw_tags if str(t).strip()]
    elif isinstance(raw_tags, str):
        tmp = raw_tags.replace("|", ",").replace(";", ",")
        tags_list = [t.strip() for t in (tmp.split(",") if "," in tmp else tmp.split()) if t.strip()]
    tags = ", ".join(tags_list) if tags_list else (raw_tags if isinstance(raw_tags, str) else "")

    # ✅ 추천 3개 강제 생성 (여기서부터 확정)
    llava_rec_list = (
        _parse_recommend_any(result.get("recommend_list"))
        or _parse_recommend_any(result.get("recommendations"))
        or _parse_recommend_any(result.get("recommend"))  # ✅ 문자열 recommend도 파싱
    )

    if len(llava_rec_list) < 3:
        rec_list = pick_top3_recs(recognized_food=food_label, goal=goal, tags_list=tags_list, k=3)
    else:
        seen = set()
        rec_list = []
        for x in llava_rec_list:
            n = _norm(x)
            if not n or n == _norm(food_label) or n in seen:
                continue
            seen.add(n)
            rec_list.append(str(x).strip())
            if len(rec_list) == 3:
                break
        if len(rec_list) < 3:
            rec_list = pick_top3_recs(recognized_food=food_label, goal=goal, tags_list=tags_list, k=3)

    # ✅ 프론트 표시 안정성: recommend는 불릿+줄바꿈, 리스트는 recommend_list
    recommend_text = "\n".join([f"- {x}" for x in rec_list])
    recommend_csv = ", ".join(rec_list)

    # ✅ 목표 적합도 간단 판정(그대로 유지)
    kcal = float((nut or {}).get("kcal") or 0)
    p = float((nut or {}).get("protein_g") or 0)
    c = float((nut or {}).get("carbs_g") or 0)
    f = float((nut or {}).get("fat_g") or 0)

    goal_l = (goal or "").lower().strip()
    fit = "OK"
    if goal_l == "cut":
        if (kcal >= 750) or (c >= 90 and p < 20):
            fit = "NOT_BEST"
    elif goal_l == "bulk":
        if (kcal <= 550) and (p < 20):
            fit = "NOT_BEST"

    goal_name = goal_l if goal_l in ["cut", "bulk"] else "maintain"

    # ✅ (PATCH) energy payload 만들기 전에 food_label을 nutrition에 주입 (곡선 음식별 차등)
    nut_for_curve = dict(nut or {})
    nut_for_curve["_food_label"] = food_label

    # ✅ 에너지 payload (start_iso 포함) + glucose_curve 포함
    energy_payload = make_energy_payload(
        nutrition=nut_for_curve,
        tags=tags_list,
        meal_type=meal_type,
        goal=goal or "",
        user_text=user_text or "",
        start_iso=final_timestamp,
    )

    glucose_metrics = (energy_payload.get("glucose_curve") or {}).get("metrics") or {}

    # ✅ (PATCH) 네가 원하는 문장 구조로 이유/요약/평가 생성
    meal_eval_override, recommend_reason_text, summary_text = build_goal_reason_text(
        food_label=food_label,
        goal=goal,
        nut=nut,
        rec_list=rec_list,
        glucose_metrics=glucose_metrics,
    )

    # ✅ 화면이 en을 렌더링하는 경우까지 커버: en도 "평가/이유/요약"을 한 번에
    peak_t = glucose_metrics.get("glucose_peak_time_min", None)
    risk = glucose_metrics.get("glycemic_risk", "")
    spike_line = ""
    if peak_t is not None and risk:
        spike_line = f"Blood sugar peak ~{int(float(peak_t))} min after meal (risk: {risk})."

    microcopy_for_en = (energy_payload.get("microcopy") or "").strip()

    en_override = (
        f"Recognized Food: {food_label} ({food_conf*100:.1f}%)\n\n"
        "Your Meal Evaluator\n"
        f"{meal_eval_override}\n\n"
        "Recommended Dishes\n"
        + "\n".join([f"- {x}" for x in rec_list]) + "\n\n"
        "Why these dishes?\n"
        + recommend_reason_text + "\n\n"
        "Summary\n"
        + summary_text + "\n\n"
        "Nutrition\n"
        + f"{food_label}: {int(kcal) if kcal else 0} kcal, {int(p)}g P / {int(f)}g F / {int(c)}g C | tags: {tags}\n\n"
        "Energy Curve Preview (next 4 hours)\n"
        + (microcopy_for_en if microcopy_for_en else "Expected curve (next 4 hours).") + "\n"
        + (spike_line if spike_line else "")
    )

    item = {
        "timestamp": final_timestamp,
        "goal": goal,
        "user_text": user_text,
        "food": food_label,
        "label": food_label,
        "food_conf": food_conf,

        "goal_score": result.get("goal_score"),
        "goal_score_text": result.get("goal_score_text", ""),

        # ✅ (PATCH) meal_eval을 우리가 만든 구조로 고정
        "meal_eval": meal_eval_override,

        # ✅ 항상 3개 추천 확정
        "recommend": recommend_text,          # 불릿+줄바꿈
        "recommend_csv": recommend_csv,       # 콤마 버전도 같이 제공(프론트가 필요하면 사용)
        "recommend_list": rec_list,           # 리스트(가장 안전)

        # ✅ (PATCH) 이유/요약도 "네 음식→목표→부족→대체안"으로 고정
        "recommend_reason": recommend_reason_text,
        "summary": summary_text,
        "en": en_override,                    # ✅ 프론트가 en 렌더링해도 구조 유지

        "nutrition": nut,
        "tags": tags,
        "manual_override": bool(manual_food),

        "height_cm": height_cm,
        "weight_kg": weight_kg,
        "bmi": bmi,
        "bmi_label": bmi_label,
        "meal_type": meal_type,
        "meal_datetime": meal_datetime,
        "server_version": VERSION,
    }

    # ✅ energy_payload 안에 glucose_curve 포함됨
    item.update(energy_payload)

    history.append(item)

    if nickname:
        user_data = load_user_data(nickname) or {"nickname": nickname, "history": []}
        user_data["history"].append(item)
        save_user_data(nickname, user_data)

    return jsonify(item)

# =========================
# ✅ History API (GET/POST)
# =========================
@app.route("/api/history", methods=["GET", "POST"])
def api_history():
    nickname = ""
    if request.method == "GET":
        nickname = (request.args.get("nickname") or "").strip()
    else:
        if request.is_json:
            payload = request.get_json(silent=True) or {}
            nickname = (payload.get("nickname") or "").strip()
        else:
            nickname = (request.form.get("nickname") or "").strip()

    if nickname:
        user_data = load_user_data(nickname) or {"history": []}
        source_list = user_data.get("history", [])
    else:
        source_list = history

    safe_list = []
    for h in source_list[-20:]:
        safe_list.append({
            "timestamp": h.get("timestamp", ""),
            "goal": h.get("goal", ""),
            "user_text": h.get("user_text", ""),
            "food": h.get("food", ""),
            "food_conf": float(h.get("food_conf", 0.0)),
            "manual_override": bool(h.get("manual_override", False)),
            "en": h.get("en", ""),

            "nutrition": h.get("nutrition") or None,
            "tags": h.get("tags", ""),

            "goal_score": h.get("goal_score"),
            "goal_score_text": h.get("goal_score_text", ""),
            "meal_eval": h.get("meal_eval", ""),

            "recommend": h.get("recommend", ""),
            "recommend_csv": h.get("recommend_csv", ""),
            "recommend_list": h.get("recommend_list", []),

            "recommend_reason": h.get("recommend_reason", ""),
            "summary": h.get("summary", ""),

            "height_cm": h.get("height_cm"),
            "weight_kg": h.get("weight_kg"),
            "bmi": h.get("bmi"),
            "bmi_label": h.get("bmi_label", ""),
            "meal_type": h.get("meal_type", ""),
            "meal_datetime": h.get("meal_datetime", ""),

            "microcopy": h.get("microcopy", ""),
            "curve": h.get("curve") or None,
            # ✅ (PATCH) history에서도 glucose_curve 내려줌
            "glucose_curve": h.get("glucose_curve") or None,
            "events": h.get("events") or [],
            "server_version": h.get("server_version", VERSION),
        })

    return Response(json.dumps(safe_list, ensure_ascii=False), mimetype="application/json")

# =========================
# ✅ History analysis API
# =========================
@app.route("/api/history_analyze", methods=["POST"])
def api_history_analyze():
    data = request.get_json(force=True) or {}
    mode = data.get("mode", "7d")
    profile = data.get("profile") or {}
    start_date_str = data.get("start_date")
    end_date_str = data.get("end_date")

    nickname = (data.get("nickname") or "").strip()
    source_list = history
    if nickname:
        user_data = load_user_data(nickname) or {"history": []}
        source_list = user_data.get("history", [])

    now = datetime.now(KST) if KST is not None else datetime.now()

    if mode == "7d":
        from_ts = now - timedelta(days=7)
        to_ts = now
        period_label = "last 7 days"
    elif mode == "30d":
        from_ts = now - timedelta(days=30)
        to_ts = now
        period_label = "last 30 days"
    elif mode == "range" and start_date_str and end_date_str:
        from_ts = datetime.fromisoformat(start_date_str + "T00:00:00")
        to_ts = datetime.fromisoformat(end_date_str + "T23:59:59")
        period_label = f"{start_date_str} to {end_date_str}"
        if KST is not None:
            if from_ts.tzinfo is None: from_ts = from_ts.replace(tzinfo=KST)
            if to_ts.tzinfo is None: to_ts = to_ts.replace(tzinfo=KST)
    else:
        from_ts = now - timedelta(days=7)
        to_ts = now
        period_label = "last 7 days (default)"

    subset = []
    for h in source_list:
        ts_str = h.get("timestamp")
        if not ts_str:
            continue
        try:
            ts = datetime.fromisoformat(ts_str)
        except Exception:
            continue
        if (from_ts <= ts <= to_ts):
            subset.append(h)

    if not subset:
        return jsonify({"text": "No meals in this period. Please log some meals first.", "period": period_label, "count": 0})

    if "llava_analyze_history_with_llava" not in globals():
        return jsonify({
            "period": period_label,
            "count": len(subset),
            "text": "llava_analyze_history_with_llava() is not defined in this notebook. Please run the cell that defines it first.",
            "raw": ""
        })

    llava_result = llava_analyze_history_with_llava(profile, subset, period_label) or {}
    return jsonify({
        "period": period_label,
        "count": len(subset),
        "text": llava_result.get("text", ""),
        "raw": llava_result.get("raw", "")
    })

# =========================
# ✅ Simple menu recommendation (no image)
# =========================
SIMPLE_MENU_DB = [
    {"name": "Spicy chicken stir-fry (dak-galbi)", "tags": ["spicy", "chicken", "high_protein", "no_seafood"], "desc": "Pan-fried chicken with vegetables and gochujang sauce."},
    {"name": "Tteokbokki (spicy rice cakes)", "tags": ["spicy", "rice", "carb", "no_seafood"], "desc": "Chewy rice cakes in a spicy red sauce."},
    {"name": "Kimchi stew with pork", "tags": ["spicy", "soup", "pork"], "desc": "Hot kimchi jjigae with pork and tofu."},
    {"name": "Bibimbap with extra vegetables", "tags": ["rice", "balanced", "veggie", "light"], "desc": "Rice bowl with mixed vegetables and a bit of meat."},
    {"name": "Grilled chicken salad", "tags": ["salad", "high_protein", "light"], "desc": "Salad topped with grilled chicken breast."},
    {"name": "Cold noodles (mul-naengmyeon)", "tags": ["noodle", "cold", "light"], "desc": "Cold buckwheat noodles in a light broth."},
    {"name": "Pork cutlet (donkatsu)", "tags": ["deep_fried", "pork", "high_calorie"], "desc": "Breaded deep-fried pork cutlet with sauce."},
    {"name": "Spicy seafood stew", "tags": ["spicy", "seafood", "soup"], "desc": "Spicy soup with assorted seafood."},
]

def score_simple_menu(menu, like_text, dislike_text, goal):
    like = (like_text or "").lower()
    dislike = (dislike_text or "").lower()
    goal = (goal or "").lower()
    tags_all = " ".join(menu["tags"]).lower() + " " + menu["name"].lower()
    score = 0

    if "spicy" in like and ("spicy" in tags_all or "kimchi" in tags_all):
        score += 3
    if ("noodle" in like or "ramen" in like or "ramyeon" in like) and "noodle" in tags_all:
        score += 2
    if ("rice" in like or "bap" in like) and "rice" in tags_all:
        score += 2
    if ("soup" in like or "stew" in like or "jjigae" in like) and "soup" in tags_all:
        score += 1
    if "salad" in like and "salad" in tags_all:
        score += 2

    if ("seafood" in dislike or "fish" in dislike or "shrimp" in dislike) and ("seafood" in tags_all or "fish" in tags_all or "shrimp" in tags_all):
        score -= 100
    if "spicy" in dislike and ("spicy" in tags_all or "kimchi" in tags_all):
        score -= 80
    if "pork" in dislike and "pork" in tags_all:
        score -= 80

    if goal == "cut":
        if "light" in menu["tags"] or "salad" in menu["tags"]:
            score += 2
        if "deep_fried" in menu["tags"] or "high_calorie" in menu["tags"]:
            score -= 2
    elif goal == "bulk":
        if "high_protein" in menu["tags"]:
            score += 2

    return score

@app.route("/api/simple_recommend", methods=["POST"])
def api_simple_recommend():
    if request.is_json:
        data = request.get_json(silent=True) or {}
        like = data.get("like", "")
        dislike = data.get("dislike", "")
        goal = data.get("goal", "")
        bmi = data.get("bmi", None)
        bmi_label = data.get("bmi_label", "")
    else:
        like = request.form.get("like", "")
        dislike = request.form.get("dislike", "")
        goal = request.form.get("goal", "")
        bmi = request.form.get("bmi", type=float)
        bmi_label = request.form.get("bmi_label", "")

    scored = sorted(((score_simple_menu(m, like, dislike, goal), m) for m in SIMPLE_MENU_DB), key=lambda x: x[0], reverse=True)
    candidates = [m for s, m in scored if s > -50]

    if not candidates:
        return jsonify({"result": "I could not find a good menu that matches your dislikes.\nMaybe choose a very simple option such as rice with vegetables or a salad and check ingredients carefully for your allergy."})

    best = candidates[0]
    top3 = candidates[:3]

    lines = [
        f"You like: {like or '(not specified)'}",
        f"You dislike / allergy: {dislike or '(not specified)'}",
    ]
    if goal:
        lines.append(f"Body goal: {goal}.")
    if bmi is not None and bmi_label:
        try:
            lines.append(f"Your BMI: {float(bmi):.1f} ({bmi_label}).")
        except Exception:
            pass

    lines += ["", "Based on this, I filtered our menu list and found these options for you:"]
    for m in top3:
        lines.append(f"- {m['name']}: {m['desc']}")

    tags_best = " ".join(best["tags"]).lower() + " " + best["name"].lower()
    reason_parts = []
    if "spicy" in (like or "").lower() and ("spicy" in tags_best or "kimchi" in tags_best):
        reason_parts.append("it matches your preference for spicy food")
    if any(x in (dislike or "").lower() for x in ["seafood", "fish", "shrimp"]) and not any(x in tags_best for x in ["seafood", "fish", "shrimp"]):
        reason_parts.append("it avoids seafood, which you said you don't like")
    if goal == "cut" and ("light" in best["tags"] or "salad" in best["tags"]):
        reason_parts.append("it is relatively light, so it fits a cutting goal")
    if goal == "bulk" and "high_protein" in best["tags"]:
        reason_parts.append("it has good protein for a bulking goal")

    reason = " and ".join(reason_parts) if reason_parts else "it is a balanced choice for you"
    lines += ["", f"Among them, I recommend {best['name']} first, because {reason}."]

    return jsonify({
        "result": "\n".join(lines),
        "recommended": best["name"],
        "candidates": [m["name"] for m in top3],
    })

# =========================
# ✅ Flask run (마지막에!)
# =========================
def run_flask():
    app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)

flask_thread = Thread(target=run_flask, daemon=True)
flask_thread.start()
print("✅ Flask started on http://127.0.0.1:5000")
print("✅ VERSION:", VERSION)


✅ Flask started on http://127.0.0.1:5000
✅ VERSION: v3rec_en_override_2025-12-15_reason_glucosefix


In [ ]:
import requests, json

url = "http://127.0.0.1:5000/api/simple_recommend"
payload = {
    "like": "spicy food",
    "dislike": "seafood",
    "goal": "cut",
    "bmi": 21.5,
    "bmi_label": "normal",
}

r = requests.post(url, json=payload)
print("status:", r.status_code)
print("raw:", r.text)


INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 00:00:01] "POST /api/simple_recommend HTTP/1.1" 200 -


status: 200
raw: {"candidates":["Kimchi stew with pork","Bibimbap with extra vegetables","Grilled chicken salad"],"recommended":"Kimchi stew with pork","result":"You like: spicy food\nYou dislike / allergy: seafood\nBody goal: cut.\nYour BMI: 21.5 (normal).\n\nBased on this, I filtered our menu list and found these options for you:\n- Kimchi stew with pork: Hot kimchi jjigae with pork and tofu.\n- Bibimbap with extra vegetables: Rice bowl with mixed vegetables and a bit of meat.\n- Grilled chicken salad: Salad topped with grilled chicken breast.\n\nAmong them, I recommend Kimchi stew with pork first, because it matches your preference for spicy food and it avoids seafood, which you said you don't like."}



In [ ]:
from threading import Thread

def run_flask():
    app.run(port=5000, debug=False, use_reloader=False)

flask_thread = Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

print("✅ Flask started on http://127.0.0.1:5000")


✅ Flask started on http://127.0.0.1:5000


Exception in thread Thread-6 (run_flask):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-502220797.py", line 4, in run_flask


In [ ]:
# === A셀: ngrok 설치 + 토큰 설정 (정답 버전) ===

# 1) ngrok 실행 파일 다운로드
!wget -q -O ngrok.zip https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok.zip
!chmod +x ngrok

# 2) ngrok v2 방식 토큰 등록 (정답)
!./ngrok authtoken "36YdOAsoc2fCiOSYZlmXvxnnwmi_6Q8badzCQzruUQrckiqa5"


Archive:  ngrok.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# pyngrok 설치
!pip install pyngrok


In [ ]:
from pyngrok import ngrok, conf

ngrok.kill()
conf.get_default().auth_token = "36YdOAsoc2fCiOSYZlmXvxnnwmi_6Q8badzCQzruUQrckiqa5"
public_url = ngrok.connect(5000, "http").public_url

print(public_url)


https://phytogenic-glinda-unlogical.ngrok-free.dev


In [ ]:
!pip install flask-cors


이미지 학습 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import numpy as np
import json
from PIL import Image

# 👇 이미지 분류기 파일 경로 (새 계정이면, 업로드한 경로로 맞춰줘)
CLS_MODEL_PATH = "/content/drive/MyDrive/ACDT_Export/food_classifier_mobilenetv2.h5"
LABELS_PATH    = "/content/drive/MyDrive/ACDT_Export/food_labels.json"

# 1) 모델 로드
food_cls_model = tf.keras.models.load_model(CLS_MODEL_PATH)

# 2) 라벨 맵 로드 (index -> 이름)
with open(LABELS_PATH, "r", encoding="utf-8") as f:
    raw_map = json.load(f)

# json에서 key가 문자열이라서, int로 바꿔줌
IDX2LABEL = {int(k): v for k, v in raw_map.items()}

# 3) 이미지 입력 크기 (학습 때 썼던 160x160)
CLS_IMG_SIZE = (160, 160)

def classify_food_from_pil(image_pil):
    """
    PIL.Image (RGB) -> (라벨 문자열, 확률)
    """
    img = image_pil.resize(CLS_IMG_SIZE)
    arr = np.array(img) / 255.0
    arr = np.expand_dims(arr, axis=0)  # (1, H, W, 3)

    preds = food_cls_model.predict(arr, verbose=0)[0]  # (num_classes,)
    top_idx = int(np.argmax(preds))
    prob = float(preds[top_idx])
    label = IDX2LABEL[top_idx]
    return label, prob


In [ ]:
from PIL import Image
import numpy as np

CLS_IMG_SIZE = (160, 160)  # 학습 때 썼던 이미지 크기에 맞춰서 (160x160이라고 가정)

def classify_food_from_pil(image_pil):
    """
    PIL.Image (RGB) -> (라벨 문자열, 확률)
    """
    img = image_pil.resize(CLS_IMG_SIZE)
    arr = np.array(img) / 255.0        # 0~1 스케일
    arr = np.expand_dims(arr, axis=0)  # (1, H, W, 3)

    preds = food_cls_model.predict(arr, verbose=0)[0]  # (num_classes,)
    top_idx = int(np.argmax(preds))
    prob = float(preds[top_idx])

    # label_map.json 구조에 따라 두 가지 케이스 처리
    if isinstance(list(IDX2LABEL.keys())[0], int):
        # {0: "ramen", 1: "kimbap", ...} 이런 형태인 경우
        label = IDX2LABEL[top_idx]
    else:
        # {"ramen": 0, "kimbap": 1, ...} 이런 형태인 경우 (안전장치)
        inv = {v: k for k, v in IDX2LABEL.items()}
        label = inv.get(top_idx, "unknown")

    return label, prob


In [ ]:
test_img_path = "/content/drive/MyDrive/final food file/Fried Egg/Img_099_0836.jpg"  # 네 이미지 경로로 바꿔줘

img = Image.open(test_img_path).convert("RGB")
label, prob = classify_food_from_pil(img)

print("예측 라벨:", label)
print("확률:", prob)


예측 라벨: Fried Egg
확률: 0.9989558458328247


A-1

In [ ]:
import pandas as pd

# 1) CSV 경로
NUTRITION_CSV_PATH = "/content/drive/MyDrive/ACDT_Export/meal_advisor_food_db_v1.csv"  # 필요하면 네 경로로 수정

# 2) CSV 읽기
nutrition_df = pd.read_csv(NUTRITION_CSV_PATH)

# 3) name_en 소문자 버전 만들어서 매칭용으로 사용
nutrition_df["name_en_lower"] = nutrition_df["name_en"].astype(str).str.strip().str.lower()

# 4) 태그(0/1)로 쓸 컬럼 리스트
BOOL_COLS = [
    "spicy", "sweet", "salty", "oily",
    "soul_food", "hangover_friendly", "late_night", "stomach_sensitive",
    "cut_friendly", "bulk_friendly", "high_protein",
    "low_carb", "low_fat", "low_sugar",
    "noodle", "rice_based", "soup_or_stew", "deep_fried", "grilled", "one_dish_meal",
    "contains_egg", "contains_milk", "contains_wheat",
    "contains_peanut", "contains_seafood", "contains_cucumber",
]

def lookup_nutrition(food_name: str):
    """
    food_name (예: 'Fried Egg') -> CSV에서 정보 찾아서 dict로 반환
    없으면 None 반환
    """
    if not food_name:
        return None

    key = food_name.strip().lower()

    # name_en_lower 기준으로 정확 매칭
    row = nutrition_df.loc[nutrition_df["name_en_lower"] == key]

    if row.empty:
        # 매칭 실패하면 None
        return None

    row = row.iloc[0]

    # 0/1 컬럼에서 값이 1인 것만 태그로 모으기
    tags = []
    for col in BOOL_COLS:
        if col in row.index:
            try:
                if int(row[col]) == 1:
                    tags.append(col)
            except (ValueError, TypeError):
                pass

    return {
        "food_name": row["name_en"],
        "category": row.get("category", ""),
        "serving_desc": row.get("serving_desc", ""),
        "kcal": float(row.get("kcal", 0)),
        "protein_g": float(row.get("protein_g", 0)),
        "fat_g": float(row.get("fat_g", 0)),
        "carbs_g": float(row.get("carbs_g", 0)),
        # 🔥 LLaVA 프롬프트에 넘길 태그 문자열
        "tags": ", ".join(tags),
    }



In [ ]:
import requests

BASE_URL = "http://127.0.0.1:5000"

try:
    r = requests.get(BASE_URL + "/api/test")
    print("Status:", r.status_code)
    print("Body:", r.json())
except Exception as e:
    print("REQUEST ERROR:", e)


REQUEST ERROR: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/test (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7d51a02efe60>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [ ]:
import requests
import os

BASE_URL = "http://127.0.0.1:5000"

# 🔁 여기 경로만 네 이미지 파일로 바꿔줘
img_path = "/content/drive/MyDrive/final food file/Fried Egg/Img_099_0003.jpg"

print("exists?", os.path.exists(img_path))

if not os.path.exists(img_path):
    raise FileNotFoundError("이미지 경로를 먼저 확인해줘!")

files = {"image": open(img_path, "rb")}
data = {
    "goal": "cut",                # 또는 "bulk", "maintain"
    "user_text": "just testing",  # 사용자 메모
    "manual_food": "",            # 여기 비워두면 classifier + LLaVA 조합 사용
}

r = requests.post(BASE_URL + "/api/recommend", files=files, data=data)
print("Status:", r.status_code)

try:
    res_json = r.json()
    print("JSON keys:", res_json.keys())
    print("food:", res_json.get("food"))
    print("food_conf:", res_json.get("food_conf"))
    print("en:", res_json.get("en"))
    print("nutrition:", res_json.get("nutrition"))
    print("tags:", res_json.get("tags"))
except Exception as e:
    print("PARSE ERROR:", e)
    print("Raw text:", r.text)


exists? True
[DEBUG] classifier food: Fried Egg 0.9466963410377502


INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 00:00:17] "POST /api/recommend HTTP/1.1" 200 -


[DEBUG] final food from LLaVA: Fried Egg
Status: 200
JSON keys: dict_keys(['bmi', 'bmi_label', 'curve', 'en', 'events', 'food', 'food_conf', 'glucose_curve', 'goal', 'goal_score', 'goal_score_text', 'height_cm', 'label', 'manual_override', 'meal_datetime', 'meal_eval', 'meal_type', 'microcopy', 'nutrition', 'recommend', 'recommend_csv', 'recommend_list', 'recommend_reason', 'server_version', 'summary', 'tags', 'timestamp', 'user_text', 'weight_kg'])
food: Fried Egg
food_conf: 0.9466963410377502
en: Recognized Food: Fried Egg (94.7%)

Your Meal Evaluator
The detected dish is **Fried Egg**. Macros are about 90 kcal · 6g P / 7g F / 0g C. Your goal is **cut**. For your goal, the key improvement is: protein is a bit low for satiety/muscle protection. Estimated blood sugar peak ~70 min after the meal (risk: LOW).

Recommended Dishes
- Egg omelette
- poached eggs
- scrambled eggs

Why these dishes?
Because your meal was **Fried Egg** and your goal is **cut**, I checked your macros first.
Your

In [ ]:
import requests

BASE_URL = "http://127.0.0.1:5000"

r = requests.get(BASE_URL + "/api/history")
print("Status:", r.status_code)

try:
    hist = r.json()
    print("History length:", len(hist))
    if hist:
        print("Last item:", hist[-1])
except Exception as e:
    print("PARSE ERROR:", e)
    print("Raw text:", r.text)


INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 00:00:33] "GET /api/history HTTP/1.1" 200 -


Status: 200
History length: 1
Last item: {'timestamp': '2025-12-15T09:00:09+09:00', 'goal': 'cut', 'user_text': 'just testing', 'food': 'Fried Egg', 'food_conf': 0.9466963410377502, 'manual_override': False, 'en': 'Recognized Food: Fried Egg (94.7%)\n\nYour Meal Evaluator\nThe detected dish is **Fried Egg**. Macros are about 90 kcal · 6g P / 7g F / 0g C. Your goal is **cut**. For your goal, the key improvement is: protein is a bit low for satiety/muscle protection. Estimated blood sugar peak ~70 min after the meal (risk: LOW).\n\nRecommended Dishes\n- Egg omelette\n- poached eggs\n- scrambled eggs\n\nWhy these dishes?\nBecause your meal was **Fried Egg** and your goal is **cut**, I checked your macros first.\nYour meal: 90 kcal · 6g P / 7g F / 0g C.\nWhat to improve for your goal: protein is a bit low for satiety/muscle protection.\nEstimated blood sugar peak ~70 min after the meal (risk: LOW).\nSo I recommended these 3 alternatives to address those points (and avoid repeating the same

simple recommend 를 고쳐보자

In [ ]:
print(app.url_map)


Map([<Rule '/static/<filename>' (HEAD, OPTIONS, GET) -> static>,
 <Rule '/api/test' (HEAD, OPTIONS, GET) -> api_test>,
 <Rule '/api/recommend' (POST, OPTIONS) -> api_recommend>,
 <Rule '/api/history' (HEAD, POST, GET, OPTIONS) -> api_history>,
 <Rule '/api/history_analyze' (POST, OPTIONS) -> api_history_analyze>,
 <Rule '/api/simple_recommend' (POST, OPTIONS) -> api_simple_recommend>])


In [ ]:
import requests, json

url = "http://127.0.0.1:5000/api/simple_recommend"
payload = {
    "like": "spicy food",
    "dislike": "seafood",
    "goal": "cut",
    "bmi": 22.0,
    "bmi_label": "normal",
}

r = requests.post(url, json=payload)
print("status:", r.status_code)
print("body:", r.text)


INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 00:00:50] "POST /api/simple_recommend HTTP/1.1" 200 -


status: 200
body: {"candidates":["Kimchi stew with pork","Bibimbap with extra vegetables","Grilled chicken salad"],"recommended":"Kimchi stew with pork","result":"You like: spicy food\nYou dislike / allergy: seafood\nBody goal: cut.\nYour BMI: 22.0 (normal).\n\nBased on this, I filtered our menu list and found these options for you:\n- Kimchi stew with pork: Hot kimchi jjigae with pork and tofu.\n- Bibimbap with extra vegetables: Rice bowl with mixed vegetables and a bit of meat.\n- Grilled chicken salad: Salad topped with grilled chicken breast.\n\nAmong them, I recommend Kimchi stew with pork first, because it matches your preference for spicy food and it avoids seafood, which you said you don't like."}

